# 3. Generating new images with COSDD

The VAE has learnt to model clean images with its latent variables and the AR decoder has learnt to model the noise generation process. This means we can generate new clean and noisy images.

In [ ]:
import os

import tifffile
import torch
import matplotlib.pyplot as plt
import numpy as np

import utils
from dvlae import DVLAE

%matplotlib inline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Part 1. Load trained model and clean and noisy data

In [ ]:
model_name = "mito-confocal"
checkpoint_path = os.path.join("checkpoints", model_name)

dvlae = DVLAE.load_from_checkpoint(os.path.join(checkpoint_path, "final_model.ckpt")).to(device)

In [ ]:
high_snr_path = "data/mito-confocal-highsnr.tif"
high_snr = tifffile.imread(high_snr_path)[:, None]

low_snr_path = "data/mito-confocal-lowsnr.tif"
low_snr = tifffile.imread(low_snr_path)[:, None]
low_snr = torch.from_numpy(low_snr.astype(float)).to(torch.float)
print(low_snr.shape)

# The high snr reference images for this dataset are on a different scale to the low snr images.
# We will scale and shift the reference images to match the noisy images.
high_snr = utils.minimise_mse(high_snr, low_snr.numpy())

### Part 2. Generating new noise for a real noisy image

First, we'll pass a noisy image to the VAE and generate a random sample from the AR decoder. This will give us another noisy image with the same underlying clean signal but a different random sample of noise.

<div class="alert alert-info">

### Task 1

Set `img_idx` to choose which image will be passed through the VAE.

</div>

In [ ]:
img_idx = ...  ### Insert an integer here
# img_idx = 0
inp_image = low_snr[img_idx:img_idx + 1].to(device)
reconstructions = dvlae.reconstruct(inp_image)
denoised = reconstructions["s_hat"].cpu()
noisy = reconstructions["x_hat"].cpu()

In [ ]:
vmin = np.percentile(low_snr.numpy(), 0.1)
vmax = np.percentile(low_snr.numpy(), 99.9)

<div class="alert alert-info">

### Task 2

Now we will look at the original noisy image and the generated noisy image. Adjust `top`, `bottom`, `left` and `right` to view different crops of the reconstructed image.

</div>

In [ ]:
top = 0
bottom = 1024
left = 0
right = 1024

crop = (0, slice(top, bottom), slice(left, right))

fig, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(low_snr[img_idx][crop], vmin=vmin, vmax=vmax)
ax[0].set_title("Original noisy image")
ax[1].imshow(high_snr[img_idx][crop], vmin=vmin, vmax=vmax)
ax[1].set_title("Reference image")
ax[2].imshow(noisy[0][crop], vmin=vmin, vmax=vmax)
ax[2].set_title("Generated noisy image")
ax[3].imshow(denoised[0][crop], vmin=vmin, vmax=vmax)
ax[3].set_title("Denoised image")

plt.show()

The spatial correlation of the generated noise can be compared to that of the real noise to get an idea of how accurate the model is. Since we have the denoised version of the generated image and the reference high snr image, we can get a noise sample by just subtracting them from their noisy versions.

In [ ]:
real_noise = low_snr[0, 0] - high_snr[0, 0]
generated_noise = noisy[0, 0] - denoised[0, 0]

real_ac = utils.autocorrelation(real_noise, max_lag=25)
generated_ac = utils.autocorrelation(generated_noise, max_lag=25)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ac1 = ax[0].imshow(real_ac, cmap="seismic", vmin=-1, vmax=1)
ax[0].set_title("Autocorrelation of real noise")
ax[0].set_xlabel("Vertical lag")
ax[0].set_ylabel("Horizontal lag")
ac2 = ax[1].imshow(generated_ac, cmap="seismic", vmin=-1, vmax=1)
ax[1].set_title("Autocorrelation of generated noise")
ax[1].set_xlabel("Vertical lag")
ax[1].set_ylabel("Horizontal lag")

fig.colorbar(ac2, fraction=0.045)
plt.show()

## 3. Generating new images

This time, we'll generate a sample from the VAE's prior and use the two decoders to reveal a new clean image and its noisy version.

In [ ]:
n_imgs = 1
reconstructions = dvlae.sample_prior(n_imgs=n_imgs)
denoised = reconstructions["s"].cpu()
noisy = reconstructions["x"].cpu()

In [ ]:
top = 0
bottom = 1024
left = 0
right = 1024

crop = (0, slice(top, bottom), slice(left, right))

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(noisy[0][crop], vmin=vmin, vmax=vmax)
ax[0].set_title("Generated noisy image")
ax[1].imshow(denoised[0][crop], vmin=vmin, vmax=vmax)
ax[1].set_title("Generated clean image")

plt.show()

<div class="alert alert-success">

### Checkpoint 3

You've finished this module on COSDD.

</div>